In [3]:
!pip install -q keras

import keras
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import datasets, layers, models, losses, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from google.colab import drive


In [1]:
from google.colab import drive
import tensorflow as tf
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -qq 'tiny-imagenet-200.zip'

Mounted at /content/drive
--2022-12-22 09:23:08--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  75.1MB/s    in 3.2s    

2022-12-22 09:23:11 (75.1 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [10]:
# input image dimensions
image_height = 64
image_width = 64
channels = 3

# Hyperparameters
batch_size = 128
num_classes = 200
epochs = 8
num_train = 100000
num_validation = 10000

In [6]:
import time
from PIL import Image
import numpy as np

path = '/content/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [np.asarray(Image.open( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i))).convert("RGB")) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(np.asarray(Image.open( path + 'val/images/{}'.format(img_name)).convert("RGB")))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)
    #return train_data, train_labels, test_data, test_labels
  
train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

starting loading data
finished loading data, in 26.994943857192993 seconds
train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


In [7]:
def shuffle_data(train_data, train_labels ):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)

    return train_data[train_idx], train_labels[train_idx]
  
train_data, train_labels = shuffle_data(train_data, train_labels)

In [8]:
# The data, shuffled and split between train and test sets:
X_train = train_data
Y_train = train_labels
X_test = test_data
Y_test = test_labels

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

In [12]:
shallow_softmax = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(64, 64, 3)),
  tf.keras.layers.Dense(200, activation='softmax'),
])
shallow_softmax.summary()

shallow_softmax.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64, 64, 200)       800       
                                                                 
Total params: 800
Trainable params: 800
Non-trainable params: 0
_________________________________________________________________
(100000, 64, 64, 3)
(100000, 200)
(10000, 64, 64, 3)
(10000, 200)


In [11]:
shallow_softmax.fit(X_train, Y_train, epochs=5)
shallow_softmax.evaluate(X_test, Y_test)

Epoch 1/5


ValueError: ignored

In [15]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    './tiny-imagenet-200/train/',
    labels = "inferred",
    label_mode = "categorical",
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (image_height, image_width),
    shuffle = True,

)

ds_test = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/tiny-imagenet-200/test/',
    labels = "inferred",
    label_mode = "categorical",
    color_mode = "rgb",
    batch_size = batch_size,
    image_size = (image_height, image_width),
    shuffle = True,
    
)

Found 100000 files belonging to 200 classes.
Found 10000 files belonging to 1 classes.


In [ ]:
shallow_softmax = tf.keras.models.Sequential([
  Input(shape=(img_height, img_width, channels)),
  Dense(200, activation='softmax'),
])
shallow_softmax.summary()

shallow_softmax.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727


In [ ]:
train_datagen = ImageDataGenerator(rescale= 1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(32, 32), color_mode='rgb', 
                                                    batch_size=256, class_mode='categorical', shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.


In [ ]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64, 64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=256, shuffle=True, seed=42)

Found 10000 validated image filenames belonging to 200 classes.


In [ ]:
shallow_softmax = tf.keras.models.Sequential([
  Input(shape=(img_height, img_width, channels)),
  Dense(200, activation='softmax'),
])
shallow_softmax.summary()

shallow_softmax.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64, 64, 200)       800       
                                                                 
Total params: 800
Trainable params: 800
Non-trainable params: 0
_________________________________________________________________


In [ ]:
shallow_softmax.fit_generator(train_generator,
                    epochs=epochs,
                    steps_per_epoch= num_train // batch_size,
                    validation_steps= num_validation // batch_size,
                    validation_data=validation_generator,
                    verbose=1)

<ipython-input-8-36f10909647a>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  shallow_softmax.fit_generator(train_generator,


Epoch 1/8


ValueError: ignored